<h1>Resnet 50 tutorial with MNIST data - Transfer learning</h1>

Transfer learning is the idea of overcoming the isolated learning paradigm and utilizing knowledge acquired for one task to solve related ones. . In Simpler terms, Transfer learning is the process of using `state-of-the-art` pretrained models such as VGG, Resnet, mobilenet, Inception for our own custom task. These models have already been pre-trained over millions of images and learnt a myriad of features from them. Adding our own dense neural network at the end results in the model fine-tuning for our task and producing much better predictions than if we had only worked with our own data. This method is helpful when we do not have a lot of data, or our task resembles other tasks that these models have been pre-trained for: 

This notebooks serves as a basic rundown/guide of using transfer learning for tasks such as image recognition. We will be using the RESNET50 model which has been trained over the `imagenet` dataset.

More information about the RESNET50 model:

https://arxiv.org/abs/1512.03385

We will be using the pre-trained model, the model weights will be imported as well in a `.h5` format:

https://www.kaggle.com/gaborfodor/keras-pretrained-models

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import imageio

from fastai import *
from fastai.vision import *
from fastai.vision.all import *

<h2>File imports</h2>

This given notebook can be used on different forms of single-channeled data, the notebook incoporates the clothing channel images and vanilla MNIST for digit recognition. The steps will be the same for any such dataset which consists of single-channel pixel values as rows of a CSV file

<h3>The Clothing single channel CSV</h3>

In [ ]:
df = pd.read_csv('../input/clothing-single-channel/fashion-mnist_train.csv')

<h3>The Vanilla MNIST CSV</h3>

In [ ]:
df = pd.read_csv('../input/digit-recognizer/train.csv')

**Reference**: The corresponding categorical labels for the numeric labels in the clothing dataset:

<ol>
    <li>T-shirt/top</li>
    <li>Trouser</li>
    <li>Pullover</li>
    <li>Dress</li>
    <li>Coat</li>
    <li>Sandal</li>
    <li>Shirt</li>
    <li>Sneaker</li>
    <li>Bag</li>
    <li>(0) Ankle boot</li>
</ol>

<h3>Sanity check for data import</h3>

In [ ]:
df.head()

In [ ]:
df.shape

<h3>Division of data and labels</h3>

We will proceed to separate out the data used for training as well as the labels, these will be separately used in our model creation phase

In [ ]:
df_x = df.loc[:,'pixel0':'pixel783']

In [ ]:
df_x

In [ ]:
df_y = df.loc[:,'label']

In [ ]:
df_y

In [ ]:
np_x = np.array(df_x)
np_x.shape

<h3>Training data reshape</h3>

As our data is in the format of `28x28` images, we will proceed to reshape them accordingly. This will result in a 3-D array with dimensions:

(Number of training samples, height, width)

In [ ]:
X_train = np.array(np_x).reshape(-1,28, 28)
y_train = np.array(df_y)

In [ ]:
print(X_train.shape)
print(y_train.shape)

<h3>Channel stacking</h3>

We will proceed to stack the data along the last dimension. This will result in an artificially created 3-channeled image (RGB). As most of the pre-trained models work on RGB images, we need to convert ours in the same format as well

In [ ]:
X_train = np.stack((X_train,)*3, axis=-1)

In [ ]:
print(X_train.shape)

<h3>Saving images</h3>

We will be using the keras method of `flow_from_directory`, this will require us to save our images in a directory. We will convert the numpy arrays of the data into `jpg` images and save them in the `/data/` directory.

In [ ]:
def save_imgs(path:Path, data, labels):
    for label in np.unique(labels):
        (path/str(label)).mkdir(parents=True,exist_ok=True)
    for i in range(len(data)):
        if(len(labels)!=0):
            imageio.imsave( str( path/str(labels[i])/(str(i)+'.jpg') ), data[i])
        else:
            imageio.imsave( str( path/(str(i)+'.jpg') ), data[i])

In [ ]:
save_imgs(Path('/data/digits'),X_train,y_train)

<h3>Sanity check</h3>

We will proceed to check the created directories to cross-verify whether our function has successfully worked:

In [ ]:
print('total classes :', len(os.listdir('/data/digits')))
print('Images with label 1: ', len(os.listdir('/data/digits/1')))

print('Image names with label 1')
print(os.listdir('/data/digits/1')[:10])

<h3>Flowing images into the Keras image generator</h3>

Here, we use the `preprocess_input` function that processes an image into a form acceptable by the RESNET50 model. We also use the `class_mode=categorical` as our labels were numeric. This allows a conversion of those labels into sparse one-hot vectors that can be used in the output neurons of our model.

In [ ]:
from keras.applications.resnet50 import preprocess_input
import tensorflow as tf

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.15)
train_generator = train_datagen.flow_from_directory('/data/digits', class_mode='categorical', subset='training')
valid_generator = train_datagen.flow_from_directory('/data/digits', class_mode='categorical', subset='validation')

<h3>Shape of x and y</h3>

We iterate over the train generator and retrieve the first element. Here, the first element is the batch of 32 images. We can then proceed to iterate over the batch and display the images. The y dimensions correspond to the labels of the data in one-hot encoding. Each y label has a sparse single dimension vector with a value of 1 corresponding to the index of the label.

In [ ]:
x, y = train_generator[0]
print(x.shape)
print(y.shape)

Our image here has pixels consisting of flat colors as we have artificially synthesized this single-channel image into a 3-channel one. Each layer of our image does not represent the `RGB` byte values but instead the single channel pixel values. This can result in the image showing varied characteristics after its conversion into RGB.

In [ ]:
plt.imshow(x[0])

We will define the path of our resnet pretrained model if we have the requirement to do so:

In [ ]:
resnet_weights_path = '../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

<h3>Model creation</h3>

We create our ResNet50 model through the `keras.applications` package. We will add the ResNet50 model and use `include_top=False` to remove the final output layer. As the ResNet model was trained for 1000 classes and we have only 10, we will replace this with our own 10 neuron layer.

As our weights for ResNet are pretrained, we will proceed to set its training capability to False. This will allow the model to only focus on the weights of the output neurons and give us a fast and efficient implementation.

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
model = Sequential()

model.add(ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet'))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model.add(Dense(10, activation = 'softmax'))

# Say not to train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(generator=train_generator,epochs=10,validation_data=valid_generator)